In [1]:
#Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV,LassoCV,ElasticNetCV
from xgboost import XGBRegressor
from sklearn.preprocessing import RobustScaler, PowerTransformer, StandardScaler, MinMaxScaler

In [2]:
#Reading train and test dataset in dataframe
train = pd.read_excel("Flight_Train.xlsx")
test = pd.read_excel("Flight_Test.xlsx")

In [4]:
#Appending both trian and test to df
df = train.append(test)

In [5]:
#Displayes first five rows of dataset by default
df.head()

,Additional_Info,Airline,Arrival_Time,Date_of_Journey,Dep_Time,Destination,Duration,Price,Route,Source,Total_Stops
0,No info,IndiGo,01:10 22 Mar,24/03/2019,22:20,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,non-stop
1,No info,Air India,13:15,1/05/2019,05:50,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,2 stops
2,No info,Jet Airways,04:25 10 Jun,9/06/2019,09:25,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,2 stops
3,No info,IndiGo,23:30,12/05/2019,18:05,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,1 stop
4,No info,IndiGo,21:35,01/03/2019,16:50,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,1 stop


In [6]:
#To check dimension
df.shape

(13354, 11)

In [7]:
#Checking for missing values, dtypes, total count, unique count
missing_value = pd.DataFrame(df.dtypes)
missing_value.rename(columns={0:'DataTypes'},inplace=True)
missing_value['Count'] = df.count()
missing_value['Unique values'] = df.nunique()
missing_value['Null Value Count'] = df.isna().sum()
missing_value['Missing value %'] = df.isna().mean().round(2)*100

In [8]:
missing_value

,DataTypes,Count,Unique values,Null Value Count,Missing value %
Additional_Info,object,13354,10,0,0.0
Airline,object,13354,12,0,0.0
Arrival_Time,object,13354,1451,0,0.0
Date_of_Journey,object,13354,44,0,0.0
Dep_Time,object,13354,223,0,0.0
Destination,object,13354,6,0,0.0
Duration,object,13354,374,0,0.0
Price,float64,10683,1870,2671,20.0
Route,object,13353,132,1,0.0
Source,object,13354,5,0,0.0


In [9]:
#Filling missing values with 'ffill' method
df['Price'] = df['Price'].fillna(method='ffill')

In [10]:
#Dropping remaining missing values as it totally 2
df.dropna(inplace=True)

In [11]:
df.head()

,Additional_Info,Airline,Arrival_Time,Date_of_Journey,Dep_Time,Destination,Duration,Price,Route,Source,Total_Stops
0,No info,IndiGo,01:10 22 Mar,24/03/2019,22:20,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,non-stop
1,No info,Air India,13:15,1/05/2019,05:50,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,2 stops
2,No info,Jet Airways,04:25 10 Jun,9/06/2019,09:25,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,2 stops
3,No info,IndiGo,23:30,12/05/2019,18:05,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,1 stop
4,No info,IndiGo,21:35,01/03/2019,16:50,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,1 stop


In [12]:
#displayes non null value count and dtype
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13353 entries, 0 to 2670
Data columns (total 11 columns):
Additional_Info    13353 non-null object
Airline            13353 non-null object
Arrival_Time       13353 non-null object
Date_of_Journey    13353 non-null object
Dep_Time           13353 non-null object
Destination        13353 non-null object
Duration           13353 non-null object
Price              13353 non-null float64
Route              13353 non-null object
Source             13353 non-null object
Total_Stops        13353 non-null object
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


# Feature Extraction

In [13]:
#We going to extract date and month from Data_of_journey and convert to Datetime Object
df['Journey_Date'] = pd.to_datetime(df["Date_of_Journey"]).dt.day

In [14]:
df['Journey_Month'] = pd.to_datetime(df["Date_of_Journey"]).dt.month

In [15]:
df.drop(['Date_of_Journey'],axis=1,inplace=True)

In [16]:
df.head()

,Additional_Info,Airline,Arrival_Time,Dep_Time,Destination,Duration,Price,Route,Source,Total_Stops,Journey_Date,Journey_Month
0,No info,IndiGo,01:10 22 Mar,22:20,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,non-stop,24,3
1,No info,Air India,13:15,05:50,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,2 stops,5,1
2,No info,Jet Airways,04:25 10 Jun,09:25,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,2 stops,6,9
3,No info,IndiGo,23:30,18:05,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,1 stop,5,12
4,No info,IndiGo,21:35,16:50,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,1 stop,3,1


In [17]:
#In Arival column we only need time not month and extract to hour and minute, changing dtype to int
df["Arrival_Time"] = df["Arrival_Time"].str.split(' ').str[0]
df["Arrival_Hour"] = df["Arrival_Time"].str.split(':').str[0].astype(int)
df["Arrival_Minute"] = df["Arrival_Time"].str.split(':').str[1].astype(int)
df.drop('Arrival_Time',axis=1,inplace=True)

In [18]:
#Extracting hour and min from Dep_Time
df['Dep_Hour'] = df['Dep_Time'].str.split(':').str[0].astype(int)
df['Dep_Minute'] = df['Dep_Time'].str.split(':').str[1].astype(int)
df.drop('Dep_Time',axis=1,inplace=True)

In [19]:
df.head()

,Additional_Info,Airline,Destination,Duration,Price,Route,Source,Total_Stops,Journey_Date,Journey_Month,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
0,No info,IndiGo,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,non-stop,24,3,1,10,22,20
1,No info,Air India,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,2 stops,5,1,13,15,5,50
2,No info,Jet Airways,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,2 stops,6,9,4,25,9,25
3,No info,IndiGo,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,1 stop,5,12,23,30,18,5
4,No info,IndiGo,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,1 stop,3,1,21,35,16,50


In [20]:
#We only need number of stops not with string
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [21]:
'''Replacing non-stop with 0 stop'''
df['Total_Stops'] = df['Total_Stops'].replace({'non-stop':'0 stop'})

In [22]:
df['Total_Stops'] = df['Total_Stops'].str.split(' ').str[0].astype(int)

In [23]:
df.head()

,Additional_Info,Airline,Destination,Duration,Price,Route,Source,Total_Stops,Journey_Date,Journey_Month,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
0,No info,IndiGo,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,0,24,3,1,10,22,20
1,No info,Air India,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,2,5,1,13,15,5,50
2,No info,Jet Airways,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,2,6,9,4,25,9,25
3,No info,IndiGo,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,1,5,12,23,30,18,5
4,No info,IndiGo,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,1,3,1,21,35,16,50


In [24]:
'''
The ‘Route’ columns mainly tell us that how many cities they have taken to reach from source to destination .
This column is very important because based on the route they took will directly effect the price of the flight So We 
split the Route column to extract the information .Regarding the ‘Nan’ values we replace those ‘Nan’ values with ‘None’
'''
df["Route_1"] = df["Route"].str.split('→ ').str[0]
df["Route_2"] = df["Route"].str.split('→ ').str[1]
df["Route_3"] = df["Route"].str.split('→ ').str[2]
df["Route_4"] = df["Route"].str.split('→ ').str[3]
df["Route_5"] = df["Route"].str.split('→ ').str[4]

In [25]:
df.head()

,Additional_Info,Airline,Destination,Duration,Price,Route,Source,Total_Stops,Journey_Date,Journey_Month,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Route_1,Route_2,Route_3,Route_4,Route_5
0,No info,IndiGo,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,0,24,3,1,10,22,20,BLR,DEL,NaN,NaN,NaN
1,No info,Air India,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,2,5,1,13,15,5,50,CCU,IXR,BBI,BLR,NaN
2,No info,Jet Airways,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,2,6,9,4,25,9,25,DEL,LKO,BOM,COK,NaN
3,No info,IndiGo,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,1,5,12,23,30,18,5,CCU,NAG,BLR,NaN,NaN
4,No info,IndiGo,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,1,3,1,21,35,16,50,BLR,NAG,DEL,NaN,NaN


In [26]:
#Filling NaN value in Route column as None
df['Route_1'].fillna('None',inplace=True)
df['Route_2'].fillna('None',inplace=True)
df['Route_3'].fillna('None',inplace=True)
df['Route_4'].fillna('None',inplace=True)
df['Route_5'].fillna('None',inplace=True)
df.drop('Route',axis=1,inplace=True)

In [27]:
df.head()

,Additional_Info,Airline,Destination,Duration,Price,Source,Total_Stops,Journey_Date,Journey_Month,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Route_1,Route_2,Route_3,Route_4,Route_5
0,No info,IndiGo,New Delhi,2h 50m,3897.0,Banglore,0,24,3,1,10,22,20,BLR,DEL,None,None,None
1,No info,Air India,Banglore,7h 25m,7662.0,Kolkata,2,5,1,13,15,5,50,CCU,IXR,BBI,BLR,None
2,No info,Jet Airways,Cochin,19h,13882.0,Delhi,2,6,9,4,25,9,25,DEL,LKO,BOM,COK,None
3,No info,IndiGo,Banglore,5h 25m,6218.0,Kolkata,1,5,12,23,30,18,5,CCU,NAG,BLR,None,None
4,No info,IndiGo,New Delhi,4h 45m,13302.0,Banglore,1,3,1,21,35,16,50,BLR,NAG,DEL,None,None


In [28]:
df.dtypes

Additional_Info     object
Airline             object
Destination         object
Duration            object
Price              float64
Source              object
Total_Stops          int32
Journey_Date         int64
Journey_Month        int64
Arrival_Hour         int32
Arrival_Minute       int32
Dep_Hour             int32
Dep_Minute           int32
Route_1             object
Route_2             object
Route_3             object
Route_4             object
Route_5             object
dtype: object

In [29]:
#Removing Duration column as we have arrival and departure
df.drop('Duration',axis=1,inplace=True)

In [30]:
df.head()

,Additional_Info,Airline,Destination,Price,Source,Total_Stops,Journey_Date,Journey_Month,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Route_1,Route_2,Route_3,Route_4,Route_5
0,No info,IndiGo,New Delhi,3897.0,Banglore,0,24,3,1,10,22,20,BLR,DEL,None,None,None
1,No info,Air India,Banglore,7662.0,Kolkata,2,5,1,13,15,5,50,CCU,IXR,BBI,BLR,None
2,No info,Jet Airways,Cochin,13882.0,Delhi,2,6,9,4,25,9,25,DEL,LKO,BOM,COK,None
3,No info,IndiGo,Banglore,6218.0,Kolkata,1,5,12,23,30,18,5,CCU,NAG,BLR,None,None
4,No info,IndiGo,New Delhi,13302.0,Banglore,1,3,1,21,35,16,50,BLR,NAG,DEL,None,None


In [31]:
#Converting categorical to numeical
df.select_dtypes(include='object').columns

Index(['Additional_Info', 'Airline', 'Destination', 'Source', 'Route_1',
       'Route_2', 'Route_3', 'Route_4', 'Route_5'],
      dtype='object')

In [32]:
def labelencode(a):
    #Lable Encoding all the categorical variable
    le=LabelEncoder()
    return le.fit_transform(a)
df['Additional_Info'] = labelencode(df['Additional_Info'])
df['Airline'] = labelencode(df['Airline'])
df['Destination'] = labelencode(df['Destination'])
df['Source'] = labelencode(df['Source'])
df['Route_1'] = labelencode(df['Route_1'])
df['Route_2'] = labelencode(df['Route_2'])
df['Route_3'] = labelencode(df['Route_3'])
df['Route_4'] = labelencode(df['Route_4'])
df['Route_5'] = labelencode(df['Route_5'])

In [33]:
df.head()

,Additional_Info,Airline,Destination,Price,Source,Total_Stops,Journey_Date,Journey_Month,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Route_1,Route_2,Route_3,Route_4,Route_5
0,8,3,5,3897.0,0,0,24,3,1,10,22,20,0,13,24,12,4
1,8,1,0,7662.0,3,2,5,1,13,15,5,50,2,25,1,3,4
2,8,4,1,13882.0,2,2,6,9,4,25,9,25,3,32,4,5,4
3,8,3,0,6218.0,3,1,5,12,23,30,18,5,2,34,3,12,4
4,8,3,5,13302.0,0,1,3,1,21,35,16,50,0,34,8,12,4


In [34]:
df.shape

(13353, 17)

In [35]:
#Train Test split
x = df.drop('Price',axis=1)
y = df['Price']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [36]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((10682, 16), (2671, 16), (10682,), (2671,))

# Model Building

In [37]:
#Build our cross validation method by taking 50 folds
kfold = KFold(n_splits=50,shuffle=True,random_state=100)

#Calculating mean RMSE score
def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model,x,y,
                                   scoring = 'neg_mean_squared_error',
                                   cv=kfold))
    return rmse

#Storing different scaler in a list to find out the best scaler for our model
scaler = [MinMaxScaler(),StandardScaler(),RobustScaler(),PowerTransformer()]

In [40]:
###LINEAR REGRESSION###
le = LinearRegression()

for i in scaler:
    lr_model = make_pipeline(i,
                     le).fit(X=x_train,y=y_train)
    print("The RMSE score by using {0} is ".format(i),cv_rmse(lr_model).mean())
warnings.filterwarnings("ignore")  

The RMSE score by using MinMaxScaler(copy=True, feature_range=(0, 1)) is  3340.2504932334377
The RMSE score by using StandardScaler(copy=True, with_mean=True, with_std=True) is  3340.2504932334377
The RMSE score by using RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) is  3340.2504932334386
The RMSE score by using PowerTransformer(copy=True, method='yeo-johnson', standardize=True) is  3335.105599157023


'''
As in Linear regression we may face overfit or underfit problem. To overcome underfitting we will increase model complexity,
where we have High Bias and Low Variance, for overfitting we will use Regularisation technique where we have Low Bias and High
Variance.
'''

In [41]:
###LASSO###

#HyperParameter Tuning
alpha = [0.00005,0.0003,0.0001,0.0005,0.0007,0.0009,0.01] #Best values 
alpha1 = [0.0005,0.0003,0.0002,0.0004,0.0001,0.0005,0.0007,0.0008]

for i in scaler:
    lasso_model1 = make_pipeline(i,LassoCV(max_iter=1e7,
                                     alphas=alpha,
                                     random_state=40)).fit(X=x_train,y=y_train)
    lasso_model2 = make_pipeline(i,LassoCV(max_iter=1e7,
                                     alphas=alpha1,
                                     random_state=40)).fit(X=x_train,y=y_train)
    
    print('The RMSE value for {0} in Lasso Model1 is '.format(i),cv_rmse(lasso_model1).mean())
    print('The RMSE value for {0} in Lasso Model2 is '.format(i),cv_rmse(lasso_model2).mean())

#warnings.filterwarnings("ignore")     

The RMSE value for MinMaxScaler(copy=True, feature_range=(0, 1)) in Lasso Model1 is  3340.247645298782
The RMSE value for MinMaxScaler(copy=True, feature_range=(0, 1)) in Lasso Model2 is  3340.250261789389
The RMSE value for StandardScaler(copy=True, with_mean=True, with_std=True) in Lasso Model1 is  3340.2497972908077
The RMSE value for StandardScaler(copy=True, with_mean=True, with_std=True) in Lasso Model2 is  3340.2504375198423
The RMSE value for RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) in Lasso Model1 is  3340.249474709656
The RMSE value for RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) in Lasso Model2 is  3340.250410623268
The RMSE value for PowerTransformer(copy=True, method='yeo-johnson', standardize=True) in Lasso Model1 is  3335.105194166887
The RMSE value for PowerTransformer(copy=True, method='yeo-johnson', standardize=True) in Lasso Model2 is  3335.1055667280903

In [42]:
# Lasso alpha value(best value)
def lasso(a):
    lasso_model = make_pipeline(PowerTransformer(),LassoCV(alphas=[a],
                               cv=kfold)).fit(x_train,y_train)
    return cv_rmse(lasso_model).mean()


for i in alpha:
    score=lasso(i)
    print(i," : ",score)

#plt.plot(alpha,alpha_score, label='Lasso')    

5e-05  :  3335.1055971283104
0.0003  :  3335.105586995334
0.0001  :  3335.1055950988493
0.0005  :  3335.1055788659933
0.0007  :  3335.105570778452
0.0009  :  3335.105562649289
0.01  :  3335.105194166887


In [43]:
###RIDGE###

#HyperParameter Tuning

alpha2=[14.5,14.6,14.7,14.8,14.9,15.0,15.0,15.2,15.3,15.4,15.5]

for i in scaler:
    ridge_model = make_pipeline(i, RidgeCV(alphas=alpha2)).fit(x_train,y_train)
    print("The RMSE value of {0} is ".format(i),cv_rmse(ridge_model).mean())

The RMSE value of MinMaxScaler(copy=True, feature_range=(0, 1)) is  3344.0724205199303
The RMSE value of StandardScaler(copy=True, with_mean=True, with_std=True) is  3340.21488858754
The RMSE value of RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) is  3340.1609759023972
The RMSE value of PowerTransformer(copy=True, method='yeo-johnson', standardize=True) is  3335.038782557765


In [44]:
#Finding the best alpha value using powertransformer

def ridge(a):
    ridge_model = make_pipeline(PowerTransformer(),RidgeCV(alphas=[a],
                               cv=kfold)).fit(x_train,y_train)
    return cv_rmse(ridge_model).mean()


for i in alpha2:
    score_r=ridge(i)
    print(i," : ",score_r)
    
#As alph value increases RMSE score is getting better    

14.5  :  3335.042442630048
14.6  :  3335.0420726957777
14.7  :  3335.04170363551
14.8  :  3335.0413354487473
14.9  :  3335.0409681349906
15.0  :  3335.0406016937427
15.0  :  3335.0406016937427
15.2  :  3335.039871426786
15.3  :  3335.0395076000846
15.4  :  3335.039144643906
15.5  :  3335.0387825577545


In [45]:
###ElasticNetCV###
def elastic():
    
    elastic = make_pipeline(PowerTransformer(),ElasticNetCV()).fit(x_train,y_train)
    return cv_rmse(elastic).mean()

elastic()#This algorithm works better for Big datasets

3705.2473288321025

In [46]:
###XGBOOST###

xgb=XGBRegressor(n_estimators=200,gamma=0,max_depth=10,objective="reg:linear")
xgb.fit(x_train,y_train)
print(cv_rmse(xgb).mean())
warnings.filterwarnings("ignore")

[19:29:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:29:20] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:29:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:29:40] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:29:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:30:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:30:13] WARNI

[19:37:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
2725.5376179446703


In [47]:
#XGBOOST is the best fot for this model
y_pred = xgb.predict(x_test)